In [1]:
import pydicom
import os
import numpy as np
import cv2
import pydicom
from glob import glob
from tqdm import tqdm
from PIL import Image
import pandas as pd
import os
import shutil
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

In [16]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val = pd.read_csv('val.csv')

### Get the original data split (train/val/test) in csv without frame sampling 

In [18]:
def group_patient(df): # Group view labels per patient
    shrinked_df = df.groupby('path').agg({
        'label': 'first',  # Unique views per patient
        'disease': 'first',  # Assumes consistent health status per patient,
        'sample_spacing': 'first',
        'start': 'first',
        'end': 'first',
        'parent_dir': 'first'
    }).reset_index()
    return shrinked_df

In [20]:
def make_csv(df , df_name = 'df', dest_dir=''):
    df.to_csv(os.path.join(dest_dir,f'{df_name}.csv'), index=False)
    print(f"saved {df_name} csv.")

In [21]:
train = group_patient(train)
val = group_patient(val)
test = group_patient(test)

make_csv(train, 'train_original')
make_csv(val, 'val_original')
make_csv(test, 'test_original')

saved train_original csv.
saved val_original csv.
saved test_original csv.
